In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
import numpy as np 
import pandas as pd 
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
import glob
import pydicom

from pydicom.pixel_data_handlers.util import apply_voi_lut
from PIL import Image

import cv2 as cv

import random 

random.seed(42)

In [ ]:
TARGET_SIZE = 512

def dicom2array(path, voi_lut = True, fix_monochrome = True):
    
    dicom = pydicom.read_file(path)
    
    
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    
    data = data - np.min(data)
    data = data / np.max(data)
    
    data = data * 255
    
    data = data.astype(np.uint8)
    
    return data

def img_vizualisation(imgs, nb_samples = 5):
    
    fig, axes = plt.subplots(nrows=nb_samples // 5, ncols=min(5, nb_samples), figsize=(min(5, nb_samples) * 4, 4 * (nb_samples // 5)))
    i = 0
    for img in imgs:
        axes[i // 5, i % 5].imshow(np.array(img), cmap=plt.cm.gray, aspect='auto')
        axes[i // 5, i % 5].axis('off')
        i += 1
    fig.show()    


In [ ]:
TRAIN_PATH = "../input/siim-covid19-detection/train/"

paths = glob.glob(TRAIN_PATH + "*/*/*.dcm")

In [ ]:
sampled_path = random.sample(paths, 10)

samples = []
for path in sampled_path:
    img = dicom2array(path)
    samples.append(img)

In [ ]:
img_vizualisation(samples, 10)

# Histogram equalization


In [ ]:
img_example = samples[4]
img_example_hist = cv.equalizeHist(img_example)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

ax1.imshow(img_example, cmap=plt.cm.gray)
ax1.axis('off')
ax1.set_title("Original image")

ax2.imshow(img_example_hist, cmap=plt.cm.gray)
ax2.axis('off')
ax2.set_title("Histogram Equalization applied on the original image")

fig.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 4))

ax1.hist(img_example.flatten(), 256, [0, 256])
ax1.set_title("Original image")

ax2.hist(img_example_hist.flatten(), 256, [0, 256])
ax2.set_title("Histogram Equalization applied on the original image")

fig.show()

### Visualization on the sample data

In [ ]:
import scipy.misc

equalized_samples = []
for sample in samples:
    img = cv.equalizeHist(sample)
    equalized_samples.append(img)
    
img_vizualisation(equalized_samples, 10)

# CLAHE (Contrast Limited Adaptive Histogram Equalization) 



### Visualization of CLAHE

In [ ]:
clahe = cv.createCLAHE(clipLimit=40.0, tileGridSize=(8,8))

img_example = samples[9]
img_example_clahe = clahe.apply(img_example)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

ax1.imshow(img_example, cmap=plt.cm.gray)
ax1.axis('off')
ax1.set_title("Original image")

ax2.imshow(img_example_clahe, cmap=plt.cm.gray)
ax2.axis('off')
ax2.set_title("CLAHE applied on the original image")

fig.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 4))

ax1.hist(img_example.flatten(), 256, [0, 256])
ax1.set_title("Original image")

ax2.hist(img_example_clahe.flatten(), 256, [0, 256])
ax2.set_title("CLAHE applied on the original image")

fig.show()

### Visualization on the sample data

In [ ]:
clahe_samples = []

clahe = cv.createCLAHE(clipLimit=40.0, tileGridSize=(8,8))

for sample in samples:
    img = clahe.apply(sample)
    clahe_samples.append(img)
    
img_vizualisation(clahe_samples, 10)

### CLAHE with different parameters

In [ ]:
clahe_samples_2 = []

clahe = cv.createCLAHE(clipLimit=20.0, tileGridSize=(15, 15))

for sample in samples:
    img = clahe.apply(sample)
    clahe_samples_2.append(img)
    
img_vizualisation(clahe_samples_2, 10)

## Contrast Enhancement of Medical X-Ray ImageUsing Morphological Operators with OptimalStructuring Element


In [ ]:
img_example = equalized_samples[9]

kernel = cv.getStructuringElement(cv.MORPH_RECT, (15, 15)) # MORPH_ELLIPSE

tophat_img = cv.morphologyEx(img_example, cv.MORPH_TOPHAT, kernel)
bothat_img = cv.morphologyEx(img_example, cv.MORPH_BLACKHAT, kernel) # Black --> Bottom

output = img_example + tophat_img - bothat_img

compare = np.concatenate((img_example, output), axis=1)

plt.figure(figsize=(20,10))
plt.imshow(compare, cmap=plt.cm.gray)
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 4))

ax1.hist(img_example.flatten(), 256, [0, 256])
ax1.set_title("Original image")

ax2.hist(output.flatten(), 256, [0, 256])
ax2.set_title("This approach on the original image")

fig.show()

In [ ]:
hat_samples = []

kernel = cv.getStructuringElement(cv.MORPH_RECT, (15, 15)) # MORPH_ELLIPSE

for sample in equalized_samples:

    tophat = cv.morphologyEx(sample, cv.MORPH_TOPHAT, kernel)
    bothat = cv.morphologyEx(sample, cv.MORPH_BLACKHAT, kernel)
    img = sample + tophat - bothat

    hat_samples.append(img)
    
img_vizualisation(hat_samples, 10)

# Noise reduction

## Median filter 
.

In [ ]:
img_example = samples[0]
noised_samples_example = cv.medianBlur(img_example, 5)

compare = np.concatenate((img_example, noised_samples_example), axis=1)

plt.figure(figsize=(20,10))
plt.imshow(compare, cmap=plt.cm.gray)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 4))

ax1.hist(img_example.flatten(), 256, [0, 256])
ax1.set_title("Original image")

ax2.hist(noised_samples_example.flatten(), 256, [0, 256])
ax2.set_title("Noise reduction on the original image")

fig.show()

## DCT-based filter


In [ ]:

from scipy.fftpack import dct, idct

def dct2(a):
    return dct(dct(a.T, norm='ortho').T, norm='ortho')

def idct2(a):
    return idct(idct(a.T, norm='ortho').T, norm='ortho')  

def dtc_transform(img):
    return idct2(dct2(img))

img_example = samples[0]
img_idct = dtc_transform(img_example)

print("Check if the image are similar :", np.allclose(img_example, img_idct))

compare = np.concatenate((img_example, img_idct), axis=1)

plt.figure(figsize=(20,10))
plt.imshow(compare, cmap=plt.cm.gray)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 4))

ax1.hist(img_example.flatten(), 256, [0, 256])
ax1.set_title("Original image")

ax2.hist(img_idct.flatten(), 256, [0, 256])
ax2.set_title("Noise reduction on the original image")

fig.show()

In [ ]:
dct_samples = []

for sample in samples:
    img = dtc_transform(sample)
    dct_samples.append(img)
    
img_vizualisation(dct_samples, 10)

# Combine the different approaches

In [ ]:
channel_1 = samples[9]
channel_2 = clahe_samples_2[9]
channel_3 = hat_samples[9]


output = np.dstack((channel_1, channel_2, channel_3))

reference = np.dstack((channel_1, channel_1, channel_1))


compare = np.concatenate((reference, output), axis=1)

plt.figure(figsize=(20,10))
plt.imshow(compare)
plt.show()

In [ ]:
multi_channel_samples = []

for i in range(len(samples)):
    channel_1 = samples[i]
    channel_2 = clahe_samples_2[i]
    # channel_2 = clahe_samples[i]
    channel_3 = hat_samples[i]
    
    out_img = np.dstack((channel_1, channel_2, channel_3))
    
    multi_channel_samples.append(out_img)
    
img_vizualisation(multi_channel_samples, 10)